In [ ]:
! pip install numpy

In [ ]:
pip show pydot


In [ ]:
pip show graphviz

#### Load dataset


In [1]:
import numpy as np
import os

def load_npz_files(directory):
    images = []
    labels = []

    for file_name in os.listdir(directory):
        if file_name.endswith(".npz"):
            file_path = os.path.join(directory, file_name)
            data = np.load(file_path)

            # Extract images and labels using the correct keys
            images.append(data['images'])  # Images are stored under the 'images' key
            labels.append(data['labels'])  # Labels are stored under the 'labels' key

    # Concatenate the images and labels from all batches
    X = np.concatenate(images, axis=0)
    y = np.concatenate(labels, axis=0)

    return X, y

# Load data from .npz files
train_dir = r"D:\BirdFeatherClassification\dataset\split_data\preprocessed_data\train"
val_dir = r"D:\BirdFeatherClassification\dataset\split_data\preprocessed_data\val"
test_dir = r"D:\BirdFeatherClassification\dataset\split_data\preprocessed_data\test"

X_train, y_train = load_npz_files(train_dir)
X_val, y_val = load_npz_files(val_dir)
X_test, y_test = load_npz_files(test_dir)

# Check the shapes of the loaded data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (23814, 128, 128, 3), y_train shape: (23814,)
X_val shape: (5096, 128, 128, 3), y_val shape: (5096,)
X_test shape: (5130, 128, 128, 3), y_test shape: (5130,)


In [ ]:
import keras
# from keras.applications import MobileNetV2
# from keras.models import Model, Sequential
from keras import layers

num_classes = 12  # Replace with the actual number of classes

# Load Pretrained MobileNetV2 without the top layers
base_model = keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze base model layers

# Global Average Pooling layer (MUST be included to flatten the output)
x = layers.GlobalAveragePooling2D()(base_model.output)  # Converts (4,4,1280) -> (1280,)

# Fully connected layer
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Custom CNN Head
custom_head = keras.Sequential([
    layers.Dense(256, activation='relu'),  # Replacing CNN layers with FC layers
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Multi-class classification
])

# Apply Custom Head
outputs = custom_head(x)

# Define Final Model
model = keras.Model(inputs=base_model.input, outputs=outputs)

# Show Model Summary
model.summary()


In [ ]:
import keras
from keras import layers, models

num_classes = 12 

# Define a custom CNN model
model = models.Sequential([
    # Convolutional Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Block 4
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten and Fully Connected Layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Regularization
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),  
    layers.Dense(num_classes, activation='softmax')  # Output Layer
])

# model summary
model.summary()


In [ ]:
import keras
import tensorflow as tf
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Graphviz\bin"
tf.keras.utils.plot_model(model)

In [6]:
from tensorflow.keras.utils import to_categorical

# One-hot encode labels for training, validation, and test sets
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)


Shape of y_train: (23814, 12)
Shape of y_val: (5096, 12)
Shape of y_test: (5130, 12)


In [7]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [8]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
from keras.callbacks import EarlyStopping

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss',  # Or 'val_accuracy'
                               patience=3,           # Number of epochs with no improvement before stopping
                               restore_best_weights=True,  # Restore model weights from the best epoch
                               verbose=1)  # Print a message when early stopping is triggered

# Fit the model with early stopping
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    batch_size=32, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])


In [10]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


161/161 ━━━━━━━━━━━━━━━━━━━━ 26s 162ms/step - accuracy: 0.9677 - loss: 0.1035
Test Accuracy: 0.9536, Test Loss: 0.1467


In [ ]:
import matplotlib.pyplot as plt

# Extract data from history
epochs = range(1, len(history.history["accuracy"]) + 1)
train_acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]

# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'bo-', label="Training Accuracy")
plt.plot(epochs, val_acc, 'ro-', label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'bo-', label="Training Loss")
plt.plot(epochs, val_loss, 'ro-', label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Assuming you have y_test (true labels) and y_pred (predicted labels)
y_pred = model.predict(X_test).argmax(axis=1)  # Convert probabilities to class indices
y_true = y_test.argmax(axis=1)  # Convert one-hot labels to class indices

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print(classification_report(y_true, y_pred))


In [13]:
# model.save("BirdFeatherClassificationTrainedModel.h5")  # Save as an HDF5 file
import os

# Define the directory
model_dir = r"D:\BirdFeatherClassification\model"

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model in the 'model' directory
model.save(os.path.join(model_dir, "BirdFeatherClassificationTrainedModel1.h5"))


In [14]:
import numpy as np
# Save training history
np.save("D:/BirdFeatherClassification/model/training_history1.npy", history.history)
